In [ ]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import Audio

In [ ]:
import numpy as np
import librosa

In [ ]:
# audio_file = '/media/Samples/SOL_0.9_HQ/Winds/Flute/ordinario/Fl-ord-C5-mf.wav'
audio_file = "/home/kureta/Music/chorales/01-[Vierstimmige Chorgesänge]-=Hilf,Gott,dass mir's gelinge=,BWV 343.mp3"

In [ ]:
y, sr = librosa.load(audio_file, sr=44100)
y = librosa.util.normalize(y)
y, sr = librosa.effects.trim(y, top_db=40, frame_length=1024, hop_length=512)
y = y[:5*44100]

In [ ]:
F = librosa.stft(y, n_fft=1024, hop_length=512)

In [ ]:
freqs = [f'{n:.2f}' for n in librosa.fft_frequencies(sr=44100, n_fft=1024)]

def major_formatter(x, pos):
    return freqs[int(x)]

In [ ]:
plt.rcParams['figure.figsize'] = (9, 4)

fig, ax1 = plt.subplots(1, 1)
ax1.set_ylim(bottom=0, top=513)
ax1.set_yticks(np.arange(0, 513, 20))
ax1.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(major_formatter))
ax1.imshow(librosa.amplitude_to_db(np.absolute(F)), aspect='auto', interpolation='none', origin='lower')
pass

In [ ]:
hF, _ = librosa.decompose.hpss(F, margin=2)

In [ ]:
harmony = librosa.istft(hF, hop_length=512)

In [ ]:
Audio(harmony, rate=44100)

In [ ]:
Audio(y, rate=44100)

In [ ]:
plt.rcParams['figure.figsize'] = (9, 4)

fig, ax1 = plt.subplots(1, 1)
ax1.plot(np.absolute(F[:, 100]))
pass

In [ ]:
from scipy.signal import find_peaks
from itertools import combinations

In [ ]:
def f0_from_stft_frame(frame):
    mean = frame.mean()
    std = frame.std()
    height = mean + std
    peak_bins = find_peaks(frame, height=height)[0]
    peak_freqs = librosa.fft_frequencies(44100, 1024)[peak_bins].astype('float32')
    diffs = [d for d in peak_freqs if librosa.midi_to_hz(35.5) <= d]

    diffs = [abs(a - b) for a, b in combinations(peak_freqs, 2)]
    diffs = [d for d in diffs if librosa.midi_to_hz(35.5) <= d <= librosa.midi_to_hz(88.5)]
    
    diffs = librosa.hz_to_midi(diffs)
    
    hist_diffs = np.histogram(diffs, range=(35.5, 88.5), bins=53)

    maxim = hist_diffs[0].argmax()
    f0 = hist_diffs[1][maxim]+0.5
    confidence = hist_diffs[0][maxim]
    
    power = frame**2
    p_mean = np.mean(power)
    p_ref = librosa.db_to_power(-20)  # or whatever other reference power you want to use
    loudness = librosa.power_to_db(p_mean, ref=p_ref)
    
    return f0, confidence, loudness

In [ ]:
Fh, _ = librosa.decompose.hpss(F, margin=16)
example = np.absolute(Fh.T)

In [ ]:
f0s = np.empty((example.shape[0],))
confidences = np.empty((example.shape[0],))
loudnesses = np.empty((example.shape[0],))
for idx, frame in enumerate(example):
    f0, confidence, loudness = f0_from_stft_frame(frame)
    f0s[idx] = f0
    confidences[idx] = confidence
    loudnesses[idx] = loudness

In [ ]:
plt.rcParams['figure.figsize'] = (9, 4)
t = np.linspace(0, len(y)/44100, len(f0s))

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
ax1.plot(t, f0s%12)
ax2.plot(t, confidences)
ax3.plot(t, loudnesses)
pass

In [ ]:
def all_f0s(frame):
    mean = frame.mean()
    std = frame.std()
    height = mean + std
    peak_bins = find_peaks(frame, height=height)[0]
    peak_freqs = librosa.fft_frequencies(44100, 1024)[peak_bins].astype('float32')

    diffs = [abs(a - b) for a, b in combinations(peak_freqs, 2)]
    diffs = [d for d in diffs if librosa.midi_to_hz(35.5) <= d <= librosa.midi_to_hz(88.5)]
    
    diffs = librosa.hz_to_midi(diffs)
    
    hist_diffs = np.histogram(diffs, range=(21.5, 108.5), bins=87)
    return hist_diffs

In [ ]:
f0s = np.empty((example.shape[0], 87))
intensities = np.empty((example.shape[0], 87))
for idx, frame in enumerate(example):
    intensity, _ = all_f0s(frame)
    intensities[idx] = intensity

In [ ]:
anan = all_f0s(example[100])

In [ ]:
plt.rcParams['figure.figsize'] = (9, 4)

fig, ax1 = plt.subplots(1, 1)

pitches = [f'{n:.0f}' for n in (anan[1] + 0.5)[:-1]]

def major_formatter(x, pos):
    return pitches[x]

ax1.set_yticks(np.arange(0, 87, 8))
ax1.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(major_formatter))
ax1.imshow(intensities.T, aspect='auto', interpolation='none', origin='lower')
pass

In [ ]:
frame = example[0]
mean = frame.mean()
std = frame.std()
height = mean + std
peak_bins = find_peaks(frame, height=height)[0]
peak_freqs = librosa.fft_frequencies(44100, 1024)[peak_bins].astype('float32')

diffs = [abs(a - b) for a, b in combinations(peak_freqs, 2)]
diffs = [d for d in diffs if librosa.midi_to_hz(21.5) <= d <= librosa.midi_to_hz(108.5)]

diffs = librosa.hz_to_midi(diffs)

fig, ax1 = plt.subplots(1, 1)
ax1.hist(diffs, range=(21.5, 108.5), bins=87, facecolor='g', alpha=0.75)
pass

In [ ]:
def stft_to_signal(S, num_iters=15):
    S_T = S.T

    # Retrieve phase information
    phase = 2 * np.pi * np.random.random_sample(S_T.shape) - np.pi
    signal = None
    for idx in range(num_iters):
        D = S_T * np.exp(1j * phase)
        signal = librosa.istft(D, hop_length=512, win_length=1024)
        # don't calculate phase during the last iteration, because it will not be used.
        if idx < num_iters - 1:
            phase = np.angle(librosa.stft(signal, n_fft=1024, hop_length=512))

    return signal

In [ ]:
sample = np.zeros((800, 513))
for i in range(800):
    sample[i] = example[0]

y = stft_to_signal(sample, 100)
Audio(data=y, rate=44100)

In [ ]:
db_spec = librosa.amplitude_to_db(np.abs(F))

In [ ]:
plt.rcParams['figure.figsize'] = (9, 4)

fig, ax1 = plt.subplots(1, 1)
ax1.set_ylim(bottom=0, top=513)
ax1.set_yticks(np.arange(0, 513, 20))
ax1.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(major_formatter))
ax1.imshow(db_spec, aspect='auto', interpolation='none', origin='lower')
pass

In [ ]:
pff = db_spec[200]
pitch_content = np.fft.rfft(pff)
pitch_content[0] = 0

In [ ]:
plt.rcParams['figure.figsize'] = (9, 4)

fig, ax1 = plt.subplots(1, 1)
ax1.plot(np.absolute(pitch_content))
pass